In [55]:
import pandas as pd
import numpy as np
import math

import tensorflow as tf
from tensorflow import keras
from keras import layers

In [13]:
# one-hot encode braid word and pad to consistent length
def encode_braid(braid, maxStrands, maxCrossings):

    b = tf.keras.utils.to_categorical(braid, num_classes=maxStrands*2)

    for i in range(maxCrossings - len(b)):
            b = np.append(b, [np.zeros(maxStrands*2)], axis=0)

    return b

In [14]:
# turn list of lists of equivalent braids with corresponding answers into trainable data
def parse_equivalent_braids(allBraids, answers, maxStrands, maxCrossings):

    x_data = []
    y_data = []

    # zip list of equivalent braids with same label
    for equivalentBraids, answer in zip(allBraids, answers):
        for braid in equivalentBraids:
            x_data.append(braid)
            y_data.append(answer)

    # encode each braid word
    x_data = [encode_braid(x, maxStrands, maxCrossings) for x in x_data]

    return np.asarray(x_data), np.asarray(y_data)

In [15]:
# read pre-parsed knot data with braid words and equivalent braids

knot_data = pd.read_csv('../data/knot_data_jones.csv')
knot_data["Equivalent Braids"] = knot_data["Equivalent Braids"].apply(eval)
knot_data["Jones"] = knot_data["Jones"].apply(eval)

In [21]:
def jones_value(coefficients, minPower, t):
    j = 0
    for i in range(len(coefficients)):
        j += coefficients[i] * (t ** (i + minPower))
    return j

In [63]:
eq = knot_data["Equivalent Braids"]

maxCrossings = max([max([len(braid) for braid in braids]) for braids in eq])
maxStrands = max([max([max([abs(b) for b in braid]) for braid in braids]) for braids in eq])

minPower = -13
maxPower = 16
nCoefficients = maxPower - minPower + 1

In [60]:
# calculate values for jones polynomial
knot_data["t=3"] = knot_data["Jones"].apply(lambda x: jones_value(x, minPower, 3))

min_val = min(knot_data["t=3"])
knot_data["t=3"] = knot_data["t=3"].apply(lambda x: math.log(x+min_val+1))

ValueError: math domain error

In [61]:
knot_data["t=3"].apply(lambda x: x + min_val+1)

0      -3.228430e+07
1      -3.228424e+07
2      -3.228586e+07
3      -3.228477e+07
4      -3.228419e+07
            ...     
2972   -3.228513e+07
2973   -3.228415e+07
2974   -3.229349e+07
2975   -3.229678e+07
2976   -2.703958e+07
Name: t=3, Length: 2977, dtype: float64

In [ ]:
math.log()

In [43]:
# split dataset for training

train_split = .8

train_data = knot_data.sample(frac=train_split)
test_data = knot_data.drop(train_data.index)

x_train, y_train = parse_equivalent_braids(train_data["Equivalent Braids"],
                    train_data["t=3"], maxStrands, maxCrossings)

x_test, y_test = parse_equivalent_braids(test_data["Equivalent Braids"],
                    test_data["t=3"], maxStrands, maxCrossings)

In [52]:
# build model from functional api

inputs = keras.Input(shape=(maxCrossings, maxStrands*2))

# recurrent version
x = layers.Masking()(inputs)
x = layers.SimpleRNN(128)(x)

# linear version
#x = layers.Flatten()(inputs)
#x = layers.Dense(64, activation="relu")(x)
#x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1)(x)

In [53]:
model = keras.Model(inputs=inputs, outputs=outputs, 
    name="jones_t3")

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_squared_error"])

model.summary()

Model: "jones_t3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 19, 12)]          0         
                                                                 
 masking_5 (Masking)         (None, 19, 12)            0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               18048     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 18,177
Trainable params: 18,177
Non-trainable params: 0
_________________________________________________________________


In [54]:
history = model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=10
)

Epoch 1/10
192/595 [========>.....................] - ETA: 1:31 - loss: 300.7761 - mean_squared_error: 300.7761

KeyboardInterrupt: 

In [ ]:
model.evaluate(x_test, y_test)

594/594 [==============================] - 2s 4ms/step - loss: 46.3204 - mean_squared_error: 46.3204


[46.320430755615234, 46.320430755615234]

In [ ]:
def predict_coefficients(braid, maxStrands, maxCrossings):
    return np.asarray(model(np.array([encode_braid(braid, maxStrands, maxCrossings)])))[0]

In [ ]:
braid = test_data.iloc[10]
word = braid["Equivalent Braids"][0]
jones = braid["Jones"]

prediction = predict_coefficients(word, maxStrands, maxCrossings)

In [ ]:
for i in range(len(jones)):
    print(jones[i], round(prediction[i]), round(prediction[i] - jones[i]))

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 -1 -1
1 1 0
-2 -3 -1
3 4 1
-5 -6 -1
6 7 1
-6 -8 -2
6 7 1
-4 -5 -1
3 4 1
-1 -2 -1
0 1 1
0 0 0
0 0 0
0 0 0
0 -1 -1
0 1 1
0 -1 -1
0 1 1
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
